# 手書き文字画像MNISTの分類(畳み込み処理)

In [2]:
#TF library
import tensorflow as tf
import tflearn


#層の作成,学習に必要なlibraryの読み込み
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression


#mnistデータセットを扱うためのlibrary
import tflearn.datasets.mnist as mnist
import numpy as np

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


## データの読み込みと前処理

In [3]:
trainX,trainY,testX,testY=mnist.load_data('./data/mnist/',one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


## データの確認

In [5]:
trainX[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [7]:
print(len(trainX[0]))#1枚目の画像ピクセル値のサイズを表示
print(trainY[0])#1枚目の画像正解データを表示

784
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


画像データは1次元配列に格納されているので2次元配列に変換

In [8]:
trainX=trainX.reshape([-1,28,28,1])
testX=testX.reshape([-1,28,28,1])

サイズの確認

In [11]:
len(trainX[0])

28

## ニューラルネットワークの作成

In [13]:
#初期化
tf.reset_default_graph()

#入力層の確認
net=input_data(shape=[None,28,28,1])

#中間層の確認
#畳み込み層の作成
net=conv_2d(net,32,5,activation='relu')
#プーリング層の作成
net=max_pool_2d(net,2)
#畳み込み層の作成
net=conv_2d(net,64,5,activation='relu')
#プーリング層の作成
net=max_pool_2d(net,2)
#全結合層の作成
net=fully_connected(net,128,activation='relu')
net=dropout(net,0.5)
#出力層の作成
net=tflearn.fully_connected(net,10,activation='softmax')
net=tflearn.regression(net,optimizer='sgd',learning_rate=0.5,loss='categorical_crossentropy')

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## モデルの生成

In [15]:
#学習の実行
model=tflearn.DNN(net)
model.fit(trainX,trainY,n_epoch=20,batch_size=100,validation_set=0.1,show_metric=True)

Training Step: 9899  | time: 100.351s
| SGD | epoch: 020 | loss: 0.00000 - acc: 0.9984 -- iter: 49400/49500
Training Step: 9900  | time: 105.474s
| SGD | epoch: 020 | loss: 0.00000 - acc: 0.9986 | val_loss: 0.03957 - val_acc: 0.9929 -- iter: 49500/49500
--


## モデルの適用

In [16]:
#作成したモデルを,テスト用データのピクセル値を格納した配列testXに適用.出力結果を変数predに格納
pred=np.array(model.predict(testX)).argmax(axis=1)
print(pred)

#テスト用データの正解の値を格納した配列testYを変数labelに格納
label=testY.argmax(axis=1)
print(label)
#predとlabelがどの程度一致しているか出力.
accuracy=np.mean(pred==label,axis=0)
print(accuracy)

[7 2 1 ... 4 5 6]
[7 2 1 ... 4 5 6]
0.9924
